In [23]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

In [24]:
es_client.info()

ObjectApiResponse({'name': '5ad71d9595ee', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'npS6rMlUQLOxgaDY5b_w0A', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [27]:
# index_name = "acw"

# # Check if the index exists, then delete it
# if es_client.indices.exists(index=index_name):
#     es_client.indices.delete(index=index_name)
#     print(f"Index '{index_name}' has been deleted.")
# else:
#     print(f"Index '{index_name}' does not exist.")

Index 'acw' has been deleted.


In [25]:
import json

with open('aegon_conquest_chunked.json', 'rt') as f_in:
    documents = json.load(f_in)

In [28]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "aegons_conquest": {"type": "text"},
            "the_dragons": {"type": "text"},
            "key_characters_and_political_developments": {"type": "text"},
            "major_battles_and_political_changes": {"type": "text"}
        }
    }
}

index_name = "acw"

# Creating the index with the corrected settings
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'acw'})

In [32]:

es_client.index(index=index_name, document=documents)

ObjectApiResponse({'_index': 'acw', '_id': 'HLdD3ZEB0QF8todLGH__', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [42]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        # "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },

                }
            }
        }
    

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [46]:
elastic_search('Meraxes')

[{'aegons_conquest': "Aegon's Conquest\nhave used Aegon’s Conquest as their touchstone for the past three\nTrue scholars know that such dating is far from precise. Aegon\nsingle day. More than two years passed between Aegon’s landing and\nhis Oldtown coronation...and even then the Conquest remained\nAegon’s reign and well into the reigns of his sons, making it impossible\nassume, wrongly, that the reign of King Aegon I Targaryen began on\nthree hills where the city of King’s Landing would eventually stand.\nNot so. The day of Aegon’s Landing was celebrated by the king and his\nOldtown by the High Septon of the Faith. This coronation took place\ntwo years after Aegon’s Landing, well after all three of the major\nbe seen that most of Aegon’s actual conquering took place from 2-1\nas Gaemon the Glorious. Gaemon’s son Aegon and his daughter Elaena\nThe Aegon who would be known to history as Aegon the Conqueror\nand Aegon the Dragon was born on Dragonstone in 27 BC. He was the\nside. Aegon 

In [ ]:
# import hashlib

# def generate_document_id(doc):
#     # combined = f"{doc['course']}-{doc['question']}"
#     combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
#     hash_object = hashlib.md5(combined.encode())
#     hash_hex = hash_object.hexdigest()
#     document_id = hash_hex[:8]
#     return document_id


# for doc in documents:
#     doc['id'] = generate_document_id(doc)